In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("arnaud58/landscape-pictures")

print("Path to dataset files:", path)

/home/gabri/miniconda3/envs/pytorch_interpreter/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 620M/620M [00:16<00:00, 39.3MB/s] 

Extracting model files...


Path to dataset files: /home/gabri/.cache/kagglehub/datasets/arnaud58/landscape-pictures/versions/2


In [3]:
import os

# Alle .jpg-Dateien im Ordner (ohne Sortierung)
files = [f for f in os.listdir(path) if f.lower().endswith(".jpg")]

total = len(files)
digits = len(str(total))  # Anzahl Ziffern

for idx, fname in enumerate(files, start=1):
    src = os.path.join(path, fname)
    new_name = f"{idx:0{digits}d}.jpg"  # z.B. 001.jpg, 002.jpg, …
    dst = os.path.join(path, new_name)
    os.rename(src, dst)

print(f"Renamed {total} files in '{path}'")

Renamed 4319 files in '/home/gabri/.cache/kagglehub/datasets/arnaud58/landscape-pictures/versions/2'
